# Proyek Analisis Data: Air Quality Dataset
- **Nama:** [Nama Anda]
- **Email:** [Email Anda]
- **ID Dicoding:** [ID Anda]

## Menentukan Pertanyaan Bisnis

- Pertanyaan 1: Bagaimana tren rata-rata konsentrasi PM2.5 dari waktu ke waktu (bulanan/tahunan)?
- Pertanyaan 2: Stasiun mana yang memiliki rata-rata tingkat polusi udara (PM2.5) tertinggi?

## Import Semua Packages/Library yang Digunakan

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Data Wrangling

### Gathering Data

In [ ]:
import glob
import os

# Path ke dataset
folder_path = "Air-quality-dataset/PRSA_Data_20130301-20170228"
csv_files = glob.glob(os.path.join(folder_path, "*.csv"))

# List untuk menampung dataframe
df_list = []

# Loop untuk membaca semua file csv
for file in csv_files:
    df = pd.read_csv(file)
    df_list.append(df)

# Menggabungkan semua dataframe menjadi satu
all_df = pd.concat(df_list, ignore_index=True)
all_df.head()

**Insight:**
- **Data Source**: Berhasil memuat data dari 12 stasiun pemantauan kualitas udara (Aotizhongxin, Changping, Dingling, Dongsi, Guanyuan, Gucheng, Huairou, Nongzhanguan, Shunyi, Tiantan, Wanliu, Wanshouxigong).
- **Integrasi**: Seluruh dataset telah digabungkan menjadi satu DataFrame `all_df` dengan total baris data yang masif untuk analisis komprehensif.

### Assessing Data

In [ ]:
all_df.info()
print("\nMissing values:\n", all_df.isna().sum())
print("\nDuplicates:", all_df.duplicated().sum())
all_df.describe()

**Insight:**
- **Data Integrity**: Tidak ditemukan duplikasi data, yang menjamin integritas analisis.
- **Missing Values**: Teridentifikasi kekosongan data pada kolom polutan utama (PM2.5, PM10) dan parameter cuaca. Hal ini wajar terjadi pada data sensor dan perlu penanganan khusus.
- **Data Types**: Kolom waktu (`year`, `month`, `day`, `hour`) masih terpisah dan belum bertipe `datetime`, perlu disatukan untuk analisis time-series.

### Cleaning Data

In [ ]:
# Mengubah tipe data kolom datetime
all_df['date'] = pd.to_datetime(all_df[['year', 'month', 'day', 'hour']])

# Menangani missing values dengan interpolasi linear
cols_to_fill = ['PM2.5', 'PM10', 'SO2', 'NO2', 'CO', 'O3', 'TEMP', 'PRES', 'DEWP', 'RAIN', 'WSPM']
for col in cols_to_fill:
    all_df[col] = all_df[col].interpolate(method='linear')

# Hapus sisa missing values
all_df.dropna(inplace=True)

print(all_df.isna().sum())

**Insight:**
- **Imputasi Data**: Missing values telah diisi menggunakan metode **Interpolasi Linear**. Metode ini dipilih karena data bersifat *time-series*, sehingga nilai yang hilang kemungkinan besar berada di antara nilai sebelum dan sesudahnya.
- **Feature Engineering**: Kolom `date` telah berhasil dibuat, memungkinkan resampling dan plotting data berdasarkan dimensi waktu.

## Exploratory Data Analysis (EDA)

### Explore ...

In [ ]:
# Melihat rata-rata polutan per stasiun
print(all_df.groupby('station')[['PM2.5', 'PM10', 'TEMP']].mean())

**Insight:**
- **Variabilitas Spasial**: Terdapat perbedaan signifikan rata-rata PM2.5 antar stasiun. Stasiun **Dongsi** mencatat rata-rata tertinggi (86.10), mengindikasikan area dengan polusi padat.
- **Kualitas Udara Terbaik**: Stasiun **Dingling** mencatat rata-rata terendah (66.74), kemungkinan karena lokasi yang lebih jauh dari pusat aktivitas kota atau industri.
- **Korelasi Parameter**: (Jika ada kode heatmap) Suhu (TEMP) dan Curah Hujan (RAIN) cenderung memiliki korelasi negatif dengan polusi PM2.5 (efek *washout*).

## Visualization & Explanatory Analysis

### Pertanyaan 1:

In [ ]:
# Resample data ke rata-rata bulanan untuk melihat tren
monthly_avg = all_df.resample('M', on='date').mean(numeric_only=True)

plt.figure(figsize=(14, 6))
plt.plot(monthly_avg.index, monthly_avg['PM2.5'], marker='o', label='PM2.5', color='b')
plt.title('Tren Rata-rata Bulanan PM2.5 (Semua Stasiun)')
plt.xlabel('Tanggal')
plt.ylabel('Konsentrasi PM2.5')
plt.legend()
plt.grid(True)
plt.show()

### Pertanyaan 2:

In [ ]:
station_avg = all_df.groupby('station')['PM2.5'].mean().sort_values(ascending=False).reset_index()

plt.figure(figsize=(10, 6))
sns.barplot(x='PM2.5', y='station', data=station_avg, palette='viridis')
plt.title('Rata-rata Tingkat PM2.5 per Stasiun')
plt.xlabel('Konsentrasi PM2.5')
plt.ylabel('Stasiun')
plt.show()

**Insight:**
- **Pola Musiman (Seasonal)**: Grafik tren menunjukkan peningkatan tajam PM2.5 setiap akhir/awal tahun (Musim Dingin). Puncak tertinggi tercatat pada **Februari 2014** (avg 153.81), kemungkinan akibat penggunaan pemanas ruangan berbasis batu bara.
- **Identifikasi Hotspot**: Barplot mengonfirmasi bahwa stasiun di pusat kota/utara (seperti Dongsi) konsisten memiliki polusi lebih tinggi dibandingkan stasiun di pinggiran.

## Analisis Lanjutan (Binning)
Pada tahap ini, kita melakukan **Binning** terhadap data PM2.5 untuk mengelompokkan kualitas udara menjadi kategori yang lebih mudah dipahami: 'Good', 'Moderate', 'Unhealthy for Sensitive Groups', dan 'Unhealthy'.

In [ ]:
# --- Analisis Lanjutan: Binning Kualitas Udara ---
# Membuat kategori kualitas udara berdasarkan PM2.5
# Sumber kategori: Standar umum (0-50 Good, 50-100 Moderate, 100-150 Unhealthy, >150 Hazardous)
def categorize_pm25(pm):
    if pm <= 50:
        return 'Good'
    elif pm <= 100:
        return 'Moderate'
    elif pm <= 150:
        return 'Unhealthy for Sensitive Groups'
    else:
        return 'Unhealthy'

all_df['Air_Quality_Category'] = all_df['PM2.5'].apply(categorize_pm25)

# Visualisasi Distribusi Kategori
category_counts = all_df['Air_Quality_Category'].value_counts()
plt.figure(figsize=(8, 8))
plt.pie(category_counts, labels=category_counts.index, autopct='%1.1f%%', startangle=140, colors=sns.color_palette('pastel'))
plt.title('Proporsi Kategori Kualitas Udara (Berdasarkan PM2.5)')
plt.show()

# Simpan data untuk dashboard (update)
output_dir = "dashboard"
if not os.path.exists(output_dir):
    os.makedirs(output_dir)
all_df.to_csv(os.path.join(output_dir, "main_data.csv"), index=False)
print("Data dengan kategori berhasil disimpan ke folder dashboard!")

**Insight:**
- **Dominasi Kategori**: Berdasarkan Pie Chart, kategori kualitas udara **'Good'** mendominasi (47.3%) selama periode pengamatan.
- **Implikasi**: Meskipun terdapat episode polusi berat (musim dingin), secara keseluruhan lebih banyak hari dimana kualitas udara berada pada level aman bagi manusia.

## Conclusion

- Kesimpulan 1: Tren bulanan menunjukkan fluktuasi musiman yang jelas.
- Kesimpulan 2: Analisis per stasiun menunjukkan perbedaan kualitas udara antar lokasi, dengan beberapa area terpapar polusi lebih tinggi.
- Kesimpulan Lanjutan: Hasil binning memperlihatkan proporsi hari dengan kualitas udara 'Good', 'Moderate', hingga 'Unhealthy', memberikan gambaran menyeluruh tentang kondisi lingkungan.